In [ ]:
import os
import sys
from open_clip import create_model_and_transforms
from open_clip import tokenize 
import torch
import numpy as np
import pandas as pd
import torch
import numpy as np
from PIL import Image
from torchvision import transforms


/root/miniconda3/envs/dac/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test_data = pd.read_csv("correct_test_set.csv")

In [3]:
test_data.head()

,image,positive,neg1,neg2,neg3,neg4
0,perspiring_59.jpg,a woman is holding a dumbbell in her hand,a woman is looking at a dumbbell in her hand,a woman is holdgrowing ong a dumbbell growing ...,a woman is holding a banana in her hand,a dancer is holding a dumbbell in her hand
1,2409207.jpg,a group of sheep in a field behind a wire fence,a group of sheep in a field wearing a wire fence,a group with sheep in a field behind a wire fence,a group of sheep in a herd behind a wire fence,a group of sheep in a field under a wire fence
2,2371196.jpg,a cat sitting on top of a television,a cat sitting using top of a televisiusing,a cat sitting standing on top of a televisista...,a cat sitting on side of a television,a woman sitting on top of a television
3,2384379.jpg,a person standing on skis in the snow,a pershas standing has skis in the snow,a perslooking at standing looking at skis in t...,a person standing on top in the snow,a person walking on skis in the snow
4,2357958.jpg,a man riding a wave on a surfboard,a man riding a wave lying on a surfboard,a man riding a wave under a surfboard,a man riding a wave off a surfboard,a man riding a bicycle on a surfboard


In [4]:
model, _, image_preprocess = create_model_and_transforms(
        "ViT-B/32",
        "openai",
        precision="amp",
        device="cpu",
        jit=False,
        force_quick_gelu=False,
        pretrained_image=False,
        image_mean=None,
        image_std=None,
        lora=4,
        freeze_img=False,
        kqv_lora=False,
    )

Model loaded
Missing Keys: []
Unexpected Keys: []


In [6]:
# checkpoint = torch.load("logs/exp_name/checkpoints/epoch_5.pt", map_location="cuda:0")
# sd = checkpoint["state_dict"]
# if next(iter(sd.items()))[0].startswith("module"):
#     sd = {k[len("module.") :]: v for k, v in sd.items()}
# model.load_state_dict(sd)
model.to("cuda")
model.eval()

CLIP(
  (visual): VisualTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0): ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (ln_attn): Identity()
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (ln): Identity()
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
        )
        (1): ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
    

In [7]:
def get_retrieval_scores_single(model, image, positive_caption, negative_caption, device="cuda"):
    # Move image to device and compute its embedding
    image_embedding = model(image.to(device), None).cpu().detach().numpy()
    image_embedding = image_embedding / np.linalg.norm(image_embedding, axis=1, keepdims=True)  # Normalize

    # Tokenize and encode the positive caption
    positive_tokenized = tokenize(positive_caption)
    positive_embedding = model(None, positive_tokenized.to(device)).cpu().detach().numpy()
    positive_embedding = positive_embedding / np.linalg.norm(positive_embedding, axis=1, keepdims=True)  # Normalize

    # Tokenize and encode the negative caption
    negative_tokenized = tokenize(negative_caption)
    negative_embedding = model(None, negative_tokenized.to(device)).cpu().detach().numpy()
    negative_embedding = negative_embedding / np.linalg.norm(negative_embedding, axis=1, keepdims=True)  # Normalize

    # Compute the similarity scores using dot product (cosine similarity)
    positive_score = np.dot(image_embedding, positive_embedding.T).item()
    negative_score = np.dot(image_embedding, negative_embedding.T).item()

    # Return the scores in a dictionary
    scores = {
        "positive_score": positive_score,
        "negative_score": negative_score
    }
    return scores

In [8]:
def get_retrieval_scores_multiple_negatives(model, image, positive_caption, negative_captions, device="cuda"):
    import numpy as np

    # Move image to device and compute its embedding
    image_embedding = model(image.to(device), None).cpu().detach().numpy()
    image_embedding = image_embedding / np.linalg.norm(image_embedding, axis=1, keepdims=True)  # Normalize

    # Tokenize and encode the positive caption
    positive_tokenized = tokenize(positive_caption)
    positive_embedding = model(None, positive_tokenized.to(device)).cpu().detach().numpy()
    positive_embedding = positive_embedding / np.linalg.norm(positive_embedding, axis=1, keepdims=True)  # Normalize

    # Compute positive score
    positive_score = np.dot(image_embedding, positive_embedding.T).item()

    # Process each negative caption
    negative_scores = {}
    for i, negative_caption in enumerate(negative_captions):
        # Tokenize and encode the negative caption
        negative_tokenized = tokenize(negative_caption)
        negative_embedding = model(None, negative_tokenized.to(device)).cpu().detach().numpy()
        negative_embedding = negative_embedding / np.linalg.norm(negative_embedding, axis=1, keepdims=True)  # Normalize

        # Compute the similarity score
        negative_scores[f"negative_score_{i+1}"] = np.dot(image_embedding, negative_embedding.T).item()

    # Combine scores into a dictionary
    scores = {
        "positive_score": positive_score,
        **negative_scores
    }

    return scores

In [10]:
def preprocess_image(image_path, device="cuda"):
    """
    Preprocesses an image from a file path.
    
    Args:
        image_path (str): Path to the image file.
        device (str): Device to move the tensor to ("cuda" or "cpu").
        
    Returns:
        torch.Tensor: Preprocessed image tensor.
    """
    # Define transformations to resize, normalize, and convert to tensor
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to match model input size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Load image, apply transformations, and move to device
    image = Image.open(image_path).convert("RGB")
    image_tensor = transform(image).unsqueeze(0)  # Add batch dimension
    return image_tensor.to(device)


def preprocess_text(caption, device="cuda"):
    """
    Preprocesses a caption string.
    
    Args:
        caption (str): The text string to be tokenized.
        device (str): Device to move the tensor to ("cuda" or "cpu").
        
    Returns:
        torch.Tensor: Tokenized text tensor.
    """
    # Tokenize the caption and convert to tensor
    caption_tokenized = tokenize(caption)
    return caption_tokenized.to(device)

In [11]:
def score_result(result_dict):
    all_scores = list(result_dict.values())

    # Sort scores in descending order
    sorted_scores = sorted(all_scores, reverse=True)

    # Find the rank of the positive score
    positive_rank = sorted_scores.index(result_dict['positive_score']) + 1  # +1 for 1-based rank

    # Check if the positive score is in top 1 or top 3
    is_top1 = positive_rank == 1
    is_top3 = positive_rank <= 3
    return is_top1, is_top3

In [12]:
top_1 = 0
top_3 = 0
positive_score = []
neg1_score = []
neg2_score = []
neg3_score = []
neg4_score = []
for idx, row in test_data.iterrows():
    try:
        image = preprocess_image("/root/autodl-fs/test_images/"+row.image, device="cuda")
        result = get_retrieval_scores_multiple_negatives(model, image, row.positive, [row.neg1, row.neg2, row.neg3, row.neg4])
        is_top_1, is_top_3 = score_result(result)
        if is_top_1:
            top_1 += 1
        if is_top_3:
            top_3 += 1
        positive_score.append(result['positive_score'])
        neg1_score.append(result['negative_score_1'])
        neg2_score.append(result['negative_score_2'])
        neg3_score.append(result['negative_score_3'])
        neg4_score.append(result['negative_score_4'])
    except Exception as e:
        print(idx)
        print(e)
        continue

In [14]:
top_1 / 1000, top_3/ 1000

(0.323, 0.877)